In [1]:
import os
import utils
from pprint import pprint
from datetime import datetime, timezone

connection_string = os.environ.get('DATABASE_URL')
db = utils.load_db(connection_string, 'travl_db')

Connected to MongoDB


## Query 0 - Full text search feature demostration on City collection

In [2]:
cursor = db.City.find(
    {'$text': {'$search': 'Boston'}}, 
    {'score': {'$meta': 'textScore'}},
).sort([('score', {'$meta': 'textScore'})])

for idx, doc in enumerate(cursor):
    print(str(idx) + ": ", end="")
    pprint(doc, indent=2)
    print()

0: { '_id': '1840000455',
  'capital': 'admin',
  'cityName': 'Boston',
  'country': { 'alpha3': 'USA',
               'countryName': 'United States',
               'currency': {'currencyName': 'US dollar', 'isoCode': 'USD'},
               'region': 'Americas'},
  'location': {'coordinates': [-71.0846, 42.3188], 'type': 'Point'},
  'population': 4688346,
  'score': 2.2}

1: { '_id': '1840023297',
  'capital': None,
  'cityName': 'Boston',
  'country': { 'alpha3': 'USA',
               'countryName': 'United States',
               'currency': {'currencyName': 'US dollar', 'isoCode': 'USD'},
               'region': 'Americas'},
  'location': {'coordinates': [-78.7555, 42.6528], 'type': 'Point'},
  'population': 8042,
  'score': 2.2}

2: { '_id': '1608800917',
  'capital': None,
  'cityName': 'Boston',
  'country': { 'alpha3': 'PHL',
               'countryName': 'Philippines',
               'currency': { 'currencyName': 'Philippine peso',
                             'isoCode': 'PHP

## Query 1 - Find all hospitals and its name, address and phone number within 50Km of the Beijing China. (_id of Bejing is 1156228865.)

In [3]:
sourceLong, sourceLat = db.City.find_one({'_id': '1156228865'})['location']['coordinates']
cursor = db.Hospital.aggregate([
    {'$geoNear': {
      'near': {'type': 'Point', 'coordinates': [sourceLong, sourceLat]},
      'distanceField': 'distance.calculated',
      'maxDistance': 50 * 1000,
      'spherical': True
    }}, {
      '$project': {
        '_id': 0, 'hospitalName': 1, 'address': 1, 'phone': 1, 'distance.calculated': 1
      }
    }, {'$sort': {'distance.calculated': 1}}
])

for idx, doc in enumerate(cursor):
    print(str(idx) + ": ", end="")
    pprint(doc, indent=2)
    print()

0: { 'address': 'No.9 Jiuxianqiaobei road,Beijing',
  'distance': {'calculated': 88.24937500451185},
  'hospitalName': 'Beijing Oasis International Hospital',
  'phone': '400-876-2747'}

1: { 'address': 'F5, Beijing New World Shopping Mall,No.3  Chong Wen Men Wai St., '
             'Beijing',
  'distance': {'calculated': 1194.300285593439},
  'hospitalName': 'Eaton Medical Group',
  'phone': '010-67085075'}

2: { 'address': 'Beijing Puhua International Hospital,No 12,Tiantan '
             'Nanli,Chongwen District,Beijing',
  'distance': {'calculated': 2428.253316162003},
  'hospitalName': 'Beijing Puhua International Hospital',
  'phone': '010-67035566'}

3: { 'address': '1F Building D,Yonggui Center,No.45 Guang Qu Men Nei St., '
             'Dongcheng District,Beijing',
  'distance': {'calculated': 3090.8062267216087},
  'hospitalName': 'Gushengtang Yonggui TCM Clinic',
  'phone': '010-67101127'}

4: { 'address': 'No. 7 Ritan East Rd, Chaoyang District, Beijing',
  'distance': {'ca

## Query 2 - Find the month with the highest new daily average Covid-19 cases in the destination country (_id of Singapore is SGP)

In [4]:
cursor = db.Country.aggregate([
    {'$match': {'_id': 'SGP'},},
    {'$unwind': {'path': '$covidCases'}},
    {'$replaceRoot': {'newRoot': '$covidCases'}},
    {'$project': {
      'year': {'$year': '$date'},
      'month': {'$month': '$date'},
      'newCases': 1,
    }},
    {'$match': {'year': 2022}},
    {'$group': {'_id': '$month', 'dailyAverageNewCases': {'$avg': '$newCases'}}},
    {'$sort': {'dailyAverageNewCases': -1}},
    {'$limit': 1},
])

for idx, doc in enumerate(cursor):
    print(str(idx) + ": ", end="")
    pprint(doc, indent=2)
    print()

0: {'_id': 2, 'dailyAverageNewCases': 13271.892857142857}



# Query 3 - Find the latest exchange rate between SGD to THB

In [5]:
cursor = db.Forex.find(
  {'currencyBase.isoCode': 'SGD', 'currencyAgainst.isoCode': 'THB'},
  {'_id': 0, 'rate': 1, 'date': 1}
).sort([('date', -1)]).limit(1)

for idx, doc in enumerate(cursor):
    print(str(idx) + ": ", end="")
    pprint(doc, indent=2)
    print()

0: {'date': datetime.datetime(2022, 10, 29, 0, 0), 'rate': 26.8978}



## Query 4 - Find information of the earliest flight from Singapore Changi Airport to Tokyo Haneda Airport for 25 November 2022

In [6]:
cursor = db.Flight.aggregate([
    {'$match': {
            'originAirport': 'WSSS', 
            'destinationAirport': 'RJTT', 
            'departureDatetime': {
                '$gte': datetime(2022, 11, 25, 0, 0, 0, tzinfo=timezone.utc),
                '$lte': datetime(2022, 11, 26, 0, 0, 0, tzinfo=timezone.utc)
            }}
    }, {'$lookup': {'from': 'Airport', 'localField': 'originAirport', 'foreignField': '_id', 'as': 'origin'}
    }, {'$lookup': {'from': 'Airport', 'localField': 'destinationAirport', 'foreignField': '_id', 'as': 'destination'}
    }, {'$unwind': {'path': '$origin'}
    }, {'$unwind': {'path': '$destination', }
    }, {'$project': {
            '_id': 0, 
            'origin.iata': 1, 
            'origin.airportName': 1, 
            'destination.iata': 1, 
            'destination.airportName': 1, 
            'departureDatetime': 1, 
            'arrivalDatetime': 1, 
            'priceUSD': 1
        }
    },
    {'$sort': {'departureDatetime': 1}},
    {'$limit': 1}
])

for idx, doc in enumerate(cursor):
    print(str(idx) + ": ", end="")
    pprint(doc, indent=2)
    print()

0: { 'arrivalDatetime': datetime.datetime(2022, 11, 25, 8, 19),
  'departureDatetime': datetime.datetime(2022, 11, 25, 1, 45),
  'destination': { 'airportName': 'Tokyo Haneda International Airport',
                   'iata': 'HND'},
  'origin': {'airportName': 'Singapore Changi Airport', 'iata': 'SIN'},
  'priceUSD': 1934.94}



## Query 5 - Get the ticket information of flight tickets that the user "bubuding" has bought, for flights in the month of Nov 2022. ("bubuding" has a _id of "cl92lvec40000obt1t62zu8ca")

In [7]:
cursor = db.User.aggregate([
    {'$match': {'_id': 'cl92lvec40000obt1t62zu8ca'}},
    {'$unwind': {'path': '$tickets'}}, 
    {'$replaceRoot': {'newRoot': '$tickets'}},
    {'$lookup': {'from': 'Flight', 'localField': 'flight', 'foreignField': '_id', 'as': 'flight'}},
    {'$lookup': {'from': 'Passenger', 'localField': 'passenger', 'foreignField': '_id', 'as': 'passenger'}}, 
    {'$unwind': {'path': '$flight'}},
    {'$unwind': {'path': '$passenger'}},
    {'$match': {'flight.departureDatetime': {
                '$gte': datetime(2022, 11, 1, 0, 0, 0, tzinfo=timezone.utc), 
                '$lt': datetime(2022, 12, 1, 0, 0, 0, tzinfo=timezone.utc)}}
    }, 
    {'$sort': {'flight.departureDatetime': 1}}
])

for idx, doc in enumerate(cursor):
  print(str(idx) + ": ", end="")
  pprint(doc, indent=2)
  print()

0: { 'class': 'First',
  'createdAt': datetime.datetime(2022, 11, 14, 1, 58, 24, 852000),
  'flight': { '_id': ObjectId('63712fba1a04abd973e87ff4'),
              'arrivalDatetime': datetime.datetime(2022, 11, 5, 3, 58),
              'balanceCapacity': 151,
              'departureDatetime': datetime.datetime(2022, 11, 4, 10, 15),
              'destinationAirport': 'SCCI',
              'originAirport': 'WSSS',
              'passengers': [ObjectId('637130401a04abd973f43e87')],
              'priceUSD': 6193.13,
              'totalCapacity': 152},
  'passenger': { '_id': ObjectId('637130401a04abd973f43e87'),
                 'age': 50,
                 'firstName': 'Matthew',
                 'flights': [ObjectId('63712fba1a04abd973e87ff4')],
                 'lastName': 'Watson',
                 'nationality': 'MYS',
                 'passportNumber': 'T7524510Y',
                 'user': 'cl92lvec40000obt1t62zu8ca'},
  'seat': '1B',
  'updatedAt': datetime.datetime(2022, 11, 14, 